In [ ]:
pip install pyLDAvis

     |████████████████████████████████| 1.7MB 4.4MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.0-py2.py3-none-any.whl size=136694 sha256=ee079cb928f698bf96874922889832ae358e1f1adc38c1b4126ac9dbe6c4a864
  Stored in directory: /root/.cache/pip/wheels/5e/62/9d/e2d11b7e09f85508b7be451fa35a2bce4b12c66c10b8bcb182
Successfully built pyLDAvis


In [ ]:
import pandas as pd
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Importing Dataset

In [ ]:
review_data= pd.read_csv("/content/sample_data/Reviews.csv")
print(review_data.head(2))
print(len(review_data))
print('Unique Products')
print(len(review_data.groupby('ProductId')))
print('Unique Users')
print(len(review_data.groupby('UserId')))

   Id  ...                                               Text
0   1  ...  I have bought several of the Vitality canned d...
1   2  ...  Product arrived labeled as Jumbo Salted Peanut...

[2 rows x 10 columns]
568454
Unique Products
74258
Unique Users
256059


Cleaning Text

In [ ]:
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Filtering and Sampling reviews

In [ ]:
review_data.dropna(axis = 0, how ='any',inplace=True) 






review_data['Text'] = review_data['Text'].apply(clean_text)
review_data['Num_words_text'] = review_data['Text'].apply(lambda x:len(str(x).split())) 

print('-------Dataset --------')
print(review_data['Score'].value_counts())
print(len(review_data))
print('-------------------------')
max_review_data_sentence_length  = review_data['Num_words_text'].max()

mask = (review_data['Num_words_text'] < 100) & (review_data['Num_words_text'] >=20)
df_short_reviews = review_data[mask]
df_sampled = df_short_reviews.groupby('Score').apply(lambda x: x.sample(n=20000)).reset_index(drop = True)

print('No of Short reviews')
print(len(df_short_reviews))



#all_sentences = train_data['text'].tolist() + test_data['text'].tolist()

-------Dataset --------
5    363111
4     80655
1     52264
3     42638
2     29743
Name: Score, dtype: int64
568411
-------------------------
No of Short reviews
373281


Removing Stopwords

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df_sampled['Text']=df_sampled['Text'].apply(remove_stopwords)

Tokenization and Lemmatization

In [ ]:

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

In [ ]:
text_list=df_sampled['Text'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

thats drinking bagsi order twice time could holding leaves impossible cant even rate benefits throw terrible terrible bags break even cold watereven holding dont order sorry late ship back would didnt open late order thinking batchnopethe containers cheaply madewhat ripe
['order', 'time', 'leave', 'impossible', 'rate', 'benefit', 'terrible', 'terrible', 'bag', 'cold', 'watereven', 'holding', 'sorry', 'late', 'ship', 'late', 'order', 'batchnopethe', 'container', 'ripe']


**Creating vocabulary dictionary and document term matrix**

In [ ]:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

In [ ]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=10, random_state=100,
                chunksize=1000, passes=50,iterations=100)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [ ]:
lda_model.print_topics()

[(0,
  '0.125*"coffee" + 0.040*"flavor" + 0.032*"good" + 0.024*"taste" + 0.024*"strong" + 0.015*"blend" + 0.015*"vanilla" + 0.015*"bean" + 0.014*"great" + 0.013*"morning"'),
 (1,
  '0.130*"food" + 0.023*"cat" + 0.021*"product" + 0.021*"good" + 0.017*"organic" + 0.016*"brand" + 0.015*"year" + 0.014*"healthy" + 0.014*"baby" + 0.012*"month"'),
 (2,
  '0.058*"water" + 0.030*"drink" + 0.029*"taste" + 0.024*"flavor" + 0.023*"bottle" + 0.017*"product" + 0.015*"good" + 0.015*"energy" + 0.013*"cold" + 0.012*"juice"'),
 (3,
  '0.075*"treat" + 0.042*"dog" + 0.021*"bread" + 0.019*"great" + 0.016*"love" + 0.016*"time" + 0.014*"little" + 0.012*"chicken" + 0.012*"flour" + 0.011*"awesome"'),
 (4,
  '0.028*"small" + 0.023*"price" + 0.023*"size" + 0.022*"good" + 0.021*"bag" + 0.019*"tea" + 0.019*"quality" + 0.017*"green" + 0.015*"brand" + 0.014*"large"'),
 (5,
  '0.041*"time" + 0.025*"cereal" + 0.020*"long" + 0.019*"minute" + 0.017*"rice" + 0.014*"year" + 0.013*"week" + 0.013*"first" + 0.012*"enjoy" + 0

Visualization of LDA Results

In [ ]:

# Visualize the topics
#https://github.com/bmabey/pyLDAvis
#https://speakerdeck.com/bmabey/visualizing-topic-models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.117466  0.027209       1        1  16.585601
8     -0.141123 -0.093642       2        1  13.318407
0      0.167265  0.029411       3        1  12.402379
6      0.008599 -0.056318       4        1   9.675128
1     -0.110567 -0.149441       5        1   8.826380
2      0.199714  0.107423       6        1   8.217909
5     -0.233838  0.193664       7        1   8.151420
4     -0.041843 -0.257718       8        1   8.149973
7      0.201900  0.024616       9        1   8.087953
3     -0.167574  0.174796      10        1   6.584851, topic_info=        Term          Freq         Total Category  logprob  loglift
2     coffee  28710.000000  28710.000000  Default  30.0000  30.0000
32      food  21181.000000  21181.000000  Default  29.0000  29.0000
176   flavor  29277.000000  29277.000000  Default  28.0000  28.0000
428    store  12631.000000  12631.000000  Default  27.0000  27.0000
328    treat   9118.000000   9118.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
357     work    817.562439   4065.071078  Topic10  -5.0030   1.1165
110  product   1263.255322  26707.114450  Topic10  -4.5679  -0.3308
112    thing    812.206497   7436.077522  Topic10  -5.0096   0.5061
11   problem    708.994829   4291.431106  Topic10  -5.1455   0.9199
612     meat    594.531302   1998.726521  Topic10  -5.3216   1.5079

[543 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
96        1  0.105416    able
96        2  0.531101    able
96        3  0.050390    able
96        5  0.012056    able
96        7  0.158588    able
...     ...       ...     ...
243       7  0.197134    year
243       8  0.024525    year
243      10  0.104348    year
3012      5  0.998148  yogurt
811       1  0.999566   yummy

[774 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 1, 7, 2, 3, 6, 5, 8, 4])